In [27]:
pwd = '/home/zjy/project/MetaIM/data'

In [21]:
import networkx as nx 
from torch_geometric.nn import MessagePassing
import torch
from torch_geometric.utils import dense_to_sparse
import torch.nn.functional as F

class SIR(MessagePassing):
    def __init__(self, N, beta, gamma, C ,steps, device):
        super().__init__(aggr="add")
        self.N = N
        self.beta = beta
        self.gamma = gamma
        self.C = C
        self.steps = steps
        self.device = device

    def init_state(self,x0=None):
        if x0 is None:
            x = torch.zeros(self.N,1).to(self.device)
            ind = torch.argsort(self.w.view(-1),descending=True)
            x[ind[:self.C]] = 1    
        else:
            x = x0    
        r = torch.zeros(self.N,1).to(self.device)
        s = 1 - x
        return s, x ,r
    
    def single_step_forward(self, edge_index, s, x, r):
        q = self.propagate(edge_index, x=torch.log(1-self.beta*x))
        q = torch.exp(q)
        s, x, r = s*q, (1-self.gamma)*x + s*(1-q) , r+ self.gamma*x
        return s, x, r

    def forward(self, edge_index, x0=None):
        s, x, r = self.init_state(x0)
        for i in range(self.steps): 
           s, x ,r = self.single_step_forward(edge_index,s,x,r)
        return s, x, r

    def message(self, x_j):
        return x_j


In [28]:
# spreading model parameters


# graph 
# generate a graph
# N = 200
# k = 6
# G = nx.barabasi_albert_graph(N,k)
# A = torch.from_numpy(nx.to_numpy_array(G))
# edge_index,_ = dense_to_sparse(A)
# degree = A.sum(dim=1)

from torch_geometric.datasets import Planetoid
from torch_geometric.utils import degree

dataset = Planetoid(root=pwd+'/cora', name='cora')
data = dataset[0]
edge_index = data.edge_index
# d = degree(edge_index[1])
# N = data.num_nodes
# # degree-greedy baseline. 
# ind = torch.argsort(d,descending=True)
# x0 = torch.zeros(N,1)
# x0[ind[:C]] = 1
# model = SIR(N,beta,gamma,C,steps)
# _,x,r = model(edge_index, x0=x0)
# print('degree-greddy baseline:',float(r.mean()))

In [23]:
N = data.num_nodes
beta = 0.15
gamma = 1.0
steps = 30
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = SIR(N,beta,gamma,50,steps,device).to(device)

In [32]:
individual_infection = torch.zeros(N,N)
for i in range(N):
    x0 = torch.zeros(N,1).to(device)
    x0[i] = 1
    _,x,r = model(edge_index.to(device), x0=x0)
    individual_infection[i] = r.squeeze().cpu()
individual_infection[0]

tensor([1.0000, 0.0240, 0.1495,  ..., 0.0000, 0.0662, 0.1429])

In [34]:
import numpy as np
np.save(pwd+'/for_meta/cora_individual_infection_sir.npy', individual_infection.numpy())

In [36]:
for t in individual_infection[0]:
    print(t)

tensor(1.)
tensor(0.0240)
tensor(0.1495)
tensor(0.)
tensor(0.0706)
tensor(0.0175)
tensor(0.0905)
tensor(0.)
tensor(0.0298)
tensor(0.0089)
tensor(0.0408)
tensor(0.0432)
tensor(0.)
tensor(0.2148)
tensor(0.1572)
tensor(0.0088)
tensor(0.0902)
tensor(0.0698)
tensor(0.1206)
tensor(0.0124)
tensor(0.1287)
tensor(0.0044)
tensor(0.0548)
tensor(0.)
tensor(0.3149)
tensor(0.0605)
tensor(0.)
tensor(0.1116)
tensor(0.0137)
tensor(0.0239)
tensor(0.1887)
tensor(0.)
tensor(0.0710)
tensor(0.0601)
tensor(0.1408)
tensor(0.0132)
tensor(0.2099)
tensor(0.0088)
tensor(0.0693)
tensor(0.0236)
tensor(0.0639)
tensor(0.1036)
tensor(0.0041)
tensor(0.0399)
tensor(0.)
tensor(0.2112)
tensor(0.0406)
tensor(0.0008)
tensor(0.1537)
tensor(0.0960)
tensor(0.0162)
tensor(0.0162)
tensor(0.0902)
tensor(0.2423)
tensor(0.0385)
tensor(0.1372)
tensor(0.0684)
tensor(0.0151)
tensor(0.0283)
tensor(0.3697)
tensor(0.0558)
tensor(0.1040)
tensor(0.0056)
tensor(9.1132e-05)
tensor(0.0725)
tensor(0.1588)
tensor(0.)
tensor(0.0043)
tensor(0.268